# Описание проекта

Вы работаете аналитиком в Яндекс.Дзене. Почти всё ваше время занимает анализ пользовательского взаимодействия с карточками статей.

Каждую карточку определяют её тема и источник (у него тоже есть тема). Примеры тем: «Красота и здоровье», «Россия», «Путешествия».

Пользователей системы характеризует возрастная категория. Скажем, «26-30» или «45+».

Есть три способа взаимодействия пользователей с системой:

- Карточка отображена для пользователя (show);
- Пользователь кликнул на карточку (click);
- Пользователь просмотрел статью карточки (view).

Каждую неделю начинающие менеджеры Денис и Валерия задают вам одни и те же вопросы:

- Cколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
- Как много событий генерируют источники с разными темами?
- Как соотносятся темы карточек и темы источников?

На шестую неделю работы вы решаете, что процесс пора автоматизировать. Для Дениса и Валерии нужно сделать дашборд.

## Для создания дашборда выполните вот эти шаги:

### Скачайте файл сертификата https://storage.yandexcloud.net/cloud-certs/CA.pem в папку с файлом .ipynb, чтобы подключиться к БД.

### Конфигурация для подключения к БД 

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'password', # пароль
            'host': 'host.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])
query = '''
     select * from dash_visits

'''

engine = create_engine(connection_string)

### Коннекция к базе, созраняем результат в таблицу `dash_visits`

In [2]:
dash_visits = pd.io.sql.read_sql(query, con = engine)

### Смотрим, что получилось

In [3]:
dash_visits.head()

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


### Проверим данные на типы, пропуски, дубликаты

In [4]:
dash_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


In [5]:
# убираем лишние пробелы в названиях колонок
dash_visits.columns = [column.strip() for column in dash_visits.columns]
dash_visits.columns

Index(['record_id', 'item_topic', 'source_topic', 'age_segment', 'dt',
       'visits'],
      dtype='object')

In [6]:
print('Кол-во дубликатов:',dash_visits.duplicated(['item_topic', 'source_topic', 'age_segment', 'dt']).sum())

Кол-во дубликатов: 0


In [7]:
for item in dash_visits.columns[1:]:
    print(f'Уникальные значения {item}')
    print(sorted(dash_visits[item].unique()))
    print('X'*30)

Уникальные значения item_topic
['Деньги', 'Дети', 'Женская психология', 'Женщины', 'Здоровье', 'Знаменитости', 'Интересные факты', 'Искусство', 'История', 'Красота', 'Культура', 'Наука', 'Общество', 'Отношения', 'Подборки', 'Полезные советы', 'Психология', 'Путешествия', 'Рассказы', 'Россия', 'Семья', 'Скандалы', 'Туризм', 'Шоу', 'Юмор']
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Уникальные значения source_topic
['Авто', 'Деньги', 'Дети', 'Еда', 'Здоровье', 'Знаменитости', 'Интерьеры', 'Искусство', 'История', 'Кино', 'Музыка', 'Одежда', 'Полезные советы', 'Политика', 'Психология', 'Путешествия', 'Ремонт', 'Россия', 'Сад и дача', 'Сделай сам', 'Семейные отношения', 'Семья', 'Спорт', 'Строительство', 'Технологии', 'Финансы']
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Уникальные значения age_segment
['18-25', '26-30', '31-35', '36-40', '41-45', '45+']
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Уникальные значения dt
[numpy.datetime64('2019-09-24T18:28:00.000000000'), numpy.datetime64('2019-09-24T18:29:00.000000000'), numpy.

### *Вывод*
В загруженных данных пропусков нет, дубликатов нет, тип данных имеет соответствующие значения, период охватываемых собыытий с 18:28:00 по 19:00:00, дата 2019-09-24. Можем записать данные в файл и продолжить работать с дашбордом

### Выгружаем данные из таблицы `dash_visits` в файл `dash_visits.csv`

In [8]:
dash_visits.to_csv('dash_visits.csv', encoding='utf-8', index=False )

In [9]:
# проверка записанного файла
dash_visits = pd.read_csv('dash_visits.csv' )
dash_visits.head()

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


### Дальше работаем в `Tableau` и `Power Point`